In [1]:
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [1]:
import cv2
import numpy as np

In [57]:
#https://github.com/spmallick/learnopencv/blob/5d9e05bce247e475488a40cfec0de11c006db532/Homography/utils.py

def mouse_handler(event, x, y, flags, data) :
    
    if event == cv2.EVENT_LBUTTONDOWN :
        cv2.circle(data['im'], (x,y),3, (0,0,255), 5, 16)
        cv2.imshow("Image", data['im']);
        if len(data['points']) < 4 :
            data['points'].append([x,y])
        
def get_four_points(im):
    
    # Set up data to send to mouse handler
    data = {}
    data['im'] = im.copy()
    data['points'] = []
    
    #Set the callback function for any mouse event
    cv2.namedWindow("Image", cv2.WINDOW_NORMAL)
    cv2.imshow("Image",im)
    cv2.setMouseCallback("Image", mouse_handler, data)
    while cv2.waitKey(0) != ord('q'):
        pass
    cv2.destroyAllWindows()
    # Convert array to np.array
    points = np.vstack(data['points']).astype(float)
    
    return points

In [2]:
"""
https://github.com/jrosebr1/imutils/blob/master/imutils/convenience.py
"""
def grab_contours(cnts):
    # if the length the contours tuple returned by cv2.findContours
    # is '2' then we are using either OpenCV v2.4, v4-beta, or
    # v4-official
    if len(cnts) == 2:
        cnts = cnts[0]

    # if the length of the contours tuple is '3' then we are using
    # either OpenCV v3, v4-pre, or v4-alpha
    elif len(cnts) == 3:
        cnts = cnts[1]

    # otherwise OpenCV has changed their cv2.findContours return
    # signature yet again and I have no idea WTH is going on
    else:
        raise Exception(("Contours tuple must have length 2 or 3, "
            "otherwise OpenCV changed their cv2.findContours return "
            "signature yet again. Refer to OpenCV's documentation "
            "in that case"))

    # return the actual contours array
    return cnts

TESTE DO OTSU PRA CROP -> HOMOGRAPHY

https://stackoverflow.com/questions/60780831/python-how-to-cut-out-an-area-with-specific-color-from-image-opencv-numpy

In [2]:
cv2.destroyAllWindows()

In [47]:
def saveVideo(frames, size=(640, 480)):
    fourcc = cv2.VideoWriter_fourcc(*'FMP4')
    out = cv2.VideoWriter('output.mp4', fourcc, 0.3, size)
    for frame in frames:
        out.write(frame)
    out.release()

In [3]:
"""
BLOB TEST
"""
import cv2
import detector_juba.contours as dj
fname = "./judd.webm"
size = (500, 1000, 3)
cap = cv2.VideoCapture(fname)
ret, frame = cap.read()

homography = dj.get_homography(frame)
im_dst = cv2.warpPerspective(frame, homography, size[:2])
im = cv2.cvtColor(im_dst, cv2.COLOR_BGR2GRAY)
detector = cv2.SimpleBlobDetector()
keypoints = detector.detect(im)
im_with_keypoints = cv2.drawKeypoints(im, keypoints, np.array([]), (0,0,255),cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)


In [1]:
import cv2 
cv2.namedWindow('Result Image', cv2.WINDOW_NORMAL)
cv2.imshow('Result Image', im_with_keypoints)
while cv2.waitKey(0) != ord('q'):
        pass
cv2.destroyAllWindows()

NameError: name 'im_with_keypoints' is not defined

mega clean == blur no output de canny

mesa branca == blur em src -> otsu no blur

In [16]:
import detector_juba.contours as dj

fname = "./judd.webm"
size = (500, 1000, 3)
cap = cv2.VideoCapture(fname)
ret, frame = cap.read()
homography = dj.get_homography(frame)
im_dst = cv2.warpPerspective(frame, homography, size[:2])

In [42]:
"""
OTSU'S METHOD TEST
"""

fname = "./judd.webm"
cap = cv2.VideoCapture(fname)
cap_size = (int(cap.get(3)), int(cap.get(4)))
ret, frame = cap.read()

src = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
dst = cv2.Canny(src, 50, 100, None, 3)
blur = cv2.GaussianBlur(src,(5,5),0)
ret3,th3 = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
cnts = cv2.findContours(th3, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    #contours, hierarchy = cv2.findContours(th3, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    #cv2.drawContours(image=image_copy, contours=contours, contourIdx=-1, color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)
image_copy2 = frame.copy()
cnts = grab_contours(cnts)
cv2.drawContours(image=image_copy2, contours=cnts, contourIdx=-1, color=(0, 0, 255), thickness=4, lineType=cv2.LINE_AA) 
image_copy = frame.copy()
image_copy3 = frame.copy()
    # Find the contour with the maximum area.
c = max(cnts, key=cv2.contourArea)
cv2.drawContours(image=image_copy3, contours=c, contourIdx=-1, color=(0, 0, 255), thickness=4, lineType=cv2.LINE_AA) 
c = cv2.approxPolyDP(c, 0.02 * cv2.arcLength(c, True), True)
cv2.drawContours(image=image_copy, contours=c, contourIdx=-1, color=(0, 0, 255), thickness=20, lineType=cv2.LINE_AA)
#lines = cv2.HoughLinesP(th3, 1, np.pi / 180, 150, None, 0, 0)
#for line in lines:
#	    x1, y1, x2, y2 = line[0]
#	    cv2.line(frame, (x1, y1), (x2, y2), (255, 0, 0), 3)
#cv2.namedWindow('Result Image', cv2.WINDOW_NORMAL)
vid_test = [frame, cv2.cvtColor(src, cv2.COLOR_GRAY2BGR), cv2.cvtColor(blur, cv2.COLOR_GRAY2BGR), cv2.cvtColor(th3, cv2.COLOR_GRAY2BGR), image_copy2, image_copy3, image_copy]
cv2.namedWindow('Result Image', cv2.WINDOW_NORMAL)
cv2.imshow("Result Image", cv2.cvtColor(th3, cv2.COLOR_GRAY2BGR))
while cv2.waitKey(0) != ord('q'):
        pass
cv2.destroyAllWindows()


In [48]:
saveVideo(vid_test, cap_size)

In [43]:
aux = cv2.copyMakeBorder(im_dst, 40, 40, 710, 710, cv2.BORDER_CONSTANT,value=[192,196,204])
cv2.namedWindow('Result Image', cv2.WINDOW_NORMAL)
cv2.imshow("Result Image", aux)
while cv2.waitKey(0) != ord('q'):
        pass
cv2.destroyAllWindows()
vid_test.append(aux)
vid_test.append(aux)

In [16]:
fname = "./judd.webm"
cap = cv2.VideoCapture(fname)
cap_size = (int(cap.get(3)), int(cap.get(4)))
#fourcc = cv2.VideoWriter_fourcc('M','J','P','G')
#outv = cv2.VideoWriter('output.avi', cv2.CAP_GSTREAMER, 20.0, (int(cap.get(3)), int(cap.get(4))))
vid_test = []
while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        print("eof")
        break

    src = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #dst = cv2.Canny(src, 50, 100, None, 3)
    blur = cv2.GaussianBlur(src,(5,5),0)
    ret3,th3 = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    image_copy = frame.copy()
    cnts = cv2.findContours(th3, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    #contours, hierarchy = cv2.findContours(th3, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    #cv2.drawContours(image=image_copy, contours=contours, contourIdx=-1, color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)
    cnts = grab_contours(cnts) 

    # Find the contour with the maximum area.
    c = max(cnts, key=cv2.contourArea)
    c = cv2.approxPolyDP(c, 0.02 * cv2.arcLength(c, True), True) 
    #cv2.drawContours(image=image_copy, contours=c, contourIdx=-1, color=(0, 0, 255), thickness=20, lineType=cv2.LINE_AA)
    vid_test.append(th3)
    # Get bounding rectangle
    #x, y, w, h = cv2.boundingRect(c)

    # Crop the bounding rectangle out of img
    #out = frame[y:y+h, x:x+w, :].copy()
    #outv.write(out.astype('uint8'))

    #lines = cv2.HoughLinesP(th3, 1, np.pi / 180, 150, None, 0, 0)
    #for line in lines:
    #        x1, y1, x2, y2 = line[0]
    #        cv2.line(frame, (x1, y1), (x2, y2), (255, 0, 0), 3)

    cv2.namedWindow('Result Image', cv2.WINDOW_NORMAL)
    cv2.imshow("Result Image", th3)
    if cv2.waitKey(1) == ord('q'):
        break
cap.release()
#outv.release(
cv2.destroyAllWindows()

eof


In [101]:
"""
AUTOMATIC TABLE COTH DELETION 
"""

import detector_juba.contours as dj

fname = "./judd.webm"
size = (500, 1000, 3)
cap = cv2.VideoCapture(fname)
ret, frame = cap.read()
homography = dj.get_homography(frame)
im_dst = cv2.warpPerspective(frame, homography, size[:2])

hsv = cv2.cvtColor(im_dst, cv2.COLOR_BGR2HSV);
h,s,v = cv2.split(hsv);
# get uniques
unique_colors, counts = np.unique(s, return_counts=True)
unique_h, counts_h = np.unique(h, return_counts=True)
unique_v, counts_v = np.unique(v, return_counts=True);

# sort through and grab the most abundant unique color
big_color, big_colorh, big_colorv = None, None, None
biggest, biggesth, biggestv = -1, -1, -1
for a in range(len(unique_colors)):
    if counts[a] > biggest:
        biggest = counts[a];
        big_color = int(unique_colors[a])
for a in range(len(unique_h)):
    if counts_h[a] > biggesth:
        biggesth = counts_h[a];
        big_colorh = int(unique_h[a])
for a in range(len(unique_v)):
    if counts_v[a] > biggestv:
        biggestv = counts_v[a];
        big_colorv = int(unique_v[a])
# get the color mask
marginh, margin, marginv = 5, 80, 120
#lowerb = np.array([52, 150, 50])
#upperb = np.array([60, 255, 220])
lowerb = np.array([big_colorh - marginh, big_color-margin, big_colorv - marginv])
upperb = np.array([big_colorh + marginh, big_color+margin, big_colorv + marginv])
mask = cv2.inRange(hsv, lowerb, upperb)

kernel = np.ones((5,5), np.uint8)
mask_closing = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel) # Cleans inner points
    
# Applies the mask to the original frame
_,mask_inv = cv2.threshold(mask_closing,5,255,cv2.THRESH_BINARY_INV)
masked_img = cv2.bitwise_and(im_dst,im_dst, mask=mask_inv)

cv2.namedWindow('Mask', cv2.WINDOW_NORMAL)
cv2.imshow("Mask", masked_img)
while cv2.waitKey(0) != ord('q'):
        pass
cv2.destroyAllWindows()

In [85]:
fname = "./judd.webm"
cap = cv2.VideoCapture(fname)

shot = []

# TODO: Automatic table corner detection
ret, frame = cap.read()
#cv2.imshow("Image", frame)
pts_src = get_four_points(frame)

In [86]:
size = (500, 1000, 3)
pts_dst = np.array(
               [
                [0,0],
                [size[0] - 1, 0],
                [size[0] - 1, size[1] -1],
                [0, size[1] - 1 ]
                ], dtype=float
               )

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        print("eof")
        break

    h, status = cv2.findHomography(pts_src, pts_dst)
    im_dst = cv2.warpPerspective(frame, h, size[:2])
    cv2.namedWindow('Image', cv2.WINDOW_NORMAL)
    cv2.imshow("Image", im_dst)
    shot.append(im_dst)
    if cv2.waitKey(1) == ord('q'):
        break

cv2.destroyAllWindows()

eof


In [87]:
cv2.imwrite("test.jpg", shot[0])

for frame in shot:
    # Smooth
    frame_blur = cv2.GaussianBlur(frame, (0,0), 2)
    frame_hsv = cv2.cvtColor(frame_blur, cv2.COLOR_BGR2HSV)

    # Snooker green color range for judd.webm
    lowerb = np.array([52, 150, 50])
    upperb = np.array([60, 255, 220])
    mask = cv2.inRange(frame_hsv, lowerb, upperb)

    kernel = np.ones((5,5), np.uint8)
    mask_closing = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel) # Cleans inner points
    
    # Applies the mask to the original frame
    _,mask_inv = cv2.threshold(mask_closing,5,255,cv2.THRESH_BINARY_INV)
    masked_img = cv2.bitwise_and(frame,frame, mask=mask_inv)

    ctrs, hierarchy = cv2.findContours(mask_inv, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    ctrs = [ctr for ctr in ctrs
                        if 300 < cv2.contourArea(ctr) < 1000
                        #and cv2.minAreaRect(ctr)[1][0]*0.28 < cv2.minAreaRect(ctr)[1][1]
                        #and cv2.minAreaRect(ctr)[1][1]*0.28 < cv2.minAreaRect(ctr)[1][0]
            ]
    frame = frame.copy()
    #draw_rectangles(ctrs, frame)
    cv2.drawContours(frame, ctrs, -1, (255, 0, 0), 3)
    cv2.namedWindow('FRAME', cv2.WINDOW_NORMAL)
    cv2.imshow("FRAME", frame)
    if cv2.waitKey(1) == ord('q'):
        break
cv2.destroyAllWindows()


In [78]:
cv2.destroyAllWindows()


In [40]:
roi = cv2.selectROI("tracker", shot[0])
cv2.destroyAllWindows()


In [46]:
tracker = cv2.TrackerMIL_create()
tracker.init(shot[0], roi)
for frame in shot:
    frame = frame.copy()
    _, roi = tracker.update(frame)
    cv2.rectangle(frame, roi, (255, 0, 0), 2, 1)
    cv2.imshow("tracker", frame)
    if cv2.waitKey(1) == ord('q'):
        break
cv2.destroyAllWindows()


error: OpenCV(4.5.3) /build/opencv/src/opencv-4.5.3/modules/video/src/tracking/detail/tracking_feature.cpp:128: error: (-215:Assertion failed) !patchSize.empty() in function 'generateRandomFeature'


In [ ]:
tracker = cv2.TrackerCSRT(shot[0], roi)


In [21]:
trackers = cv2.legacy.MultiTracker_create()


AttributeError: module 'cv2' has no attribute 'MultiTracker'